In [1]:
import numpy as np
import pandas as pd

from datetime import datetime
import os
import glob

import random
from collections import deque

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing



%matplotlib inline

In [2]:
path = os.path.join("D:",os.sep, "Datasets","Stock","h1")#, "*.csv")
# os.listdir(path)

    # read all data into 1 dataframed
df_list = []
for (root, dirs ,files) in os.walk(path):
    # df_list = (pd.read_csv(os.path.join(path,f), delimiter='\t') for file in files)
    # big_df= pd.concat(df_list, ignore_index=True)
    for f in files:
        df_tmp = pd.read_csv(os.path.join(path,f), delimiter='\t')
        df_tmp['Instrument'] = f.split('_')[0]
        df_list.append(df_tmp)
        print(f.split('_')[0] + ' has updated!')

df= pd.concat(df_list, ignore_index=True)

# read all data into 1 dataframed
df_list = []
for (root, dirs ,files) in os.walk(path):
    # df_list = (pd.read_csv(os.path.join(path,f), delimiter='\t') for file in files)
    # big_df= pd.concat(df_list, ignore_index=True)
    for f in files:
        df_tmp = pd.read_csv(os.path.join(path,f), delimiter='\t')
        df_tmp['Instrument'] = f.split('_')[0]
        df_list.append(df_tmp)
        print(f.split('_')[0] + ' has updated!')

df= pd.concat(df_list, ignore_index=True)

df.to_csv('../Dataset/stock.csv', index=None)

In [3]:
# read all data and create low price dataframe into 1 dataframed
df= pd.DataFrame()
for (root, dirs ,files) in os.walk(path):
    # df_list = (pd.read_csv(os.path.join(path,f), delimiter='\t') for file in files)
    # big_df= pd.concat(df_list, ignore_index=True)
    for f in files:
        df_tmp = pd.read_csv(os.path.join(path,f), delimiter='\t')
        Instrument = f.split('_')[0] 
        Instrument = Instrument[:3] + '-' + Instrument[3:]
        df_tmp.set_index('Time', inplace=True)
        
        df_tmp.rename(columns={"Close":f"{Instrument}_Close", "Volume":f"{Instrument}_Volume"}, inplace=True)
        df_tmp = df_tmp[[f"{Instrument}_Close", f"{Instrument}_Volume"]]
        
        
        df = df.join(df_tmp, how='right')
        
        
        print(Instrument + ' has updated!')
    

df.fillna(method='ffill', inplace=True)
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [5]:
df.isna().sum()
# Visually check for null values
# sns.heatmap(df.isna())

Series([], dtype: float64)

In [6]:
df.columns

Index([], dtype='object')

In [12]:
df

,Time,Open,High,Low,Close,Volume,EURGBP_Low,EURUSD_Low,GBPUSD_Low,USDJPY_Low
0,2007-07-24 18:00:00,0.67010,0.6701,0.66966,0.67000,15421,0.66966,NaN,NaN,NaN
1,2007-07-24 19:00:00,0.67000,0.6707,0.66985,0.67045,84904,0.66985,NaN,NaN,NaN
2,2007-07-24 20:00:00,0.67045,0.6706,0.66990,0.67010,33690,0.66990,NaN,NaN,NaN
3,2007-07-24 21:00:00,0.67010,0.6707,0.66990,0.67050,8960,0.66990,NaN,NaN,NaN
4,2007-07-24 22:00:00,0.67050,0.6707,0.67032,0.67040,16738,0.67032,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
400042,2023-08-01 00:00:00,142.32400,142.5010,142.25400,142.49600,11202,NaN,NaN,NaN,142.254
400043,2023-08-01 01:00:00,142.49400,142.7840,142.43400,142.75300,14572,NaN,NaN,NaN,142.434
400044,2023-08-01 02:00:00,142.76500,142.7960,142.53200,142.57300,11623,NaN,NaN,NaN,142.532
400045,2023-08-01 03:00:00,142.57400,142.7260,142.53800,142.66700,7933,NaN,NaN,NaN,142.538


In [11]:
df[['EUR-GBP_Close', 'EUR-USD_Close', 'GBP-USD_Close']].plot()

KeyError: "None of [Index(['EUR-GBP_Close', 'EUR-USD_Close', 'GBP-USD_Close'], dtype='object')] are in the [columns]"

In [9]:
# sequence length (timeframe length)
period_len = 60
# step which predict after timefreame has trained (arbitrary value)
next_item_predict = 3
# Instrument to predict
predict_ratio = 'GBP-USD'

In [10]:
# choose next nth step as future value to predict and concat to df
# its a dummy variable just to use in calculation
df['future'] = df[f'{predict_ratio}_Close'].shift(-next_item_predict)

In [11]:
# determine if price goes up(expensive) or not
# this function is simple as returns 1 in case of price increament which means take buy position and vice versa
# we may develop the function as return differentiaion value, so the problem must consideredc as regression

def price_classification(current, future):
    if float(current) > float(future):
        return 0 
    else:
        return 1
    

In [12]:
df['target'] = list(map(price_classification, df[f'{predict_ratio}_Close'], df['future']))

## Split train and test data

In [13]:
time_steps = sorted(df.index.values)
# last 5% of timesteps
last_5_pcent = sorted(df.index.values)[-int(0.05 * len(time_steps))]

#create validation data, which index is gt last 5 percent of data
validation_df = df[(df.index) >= last_5_pcent]
#main data turns to first 95% of timeframes
df =  df[(df.index) < last_5_pcent]

In [14]:
# df.columns
def preproc_data(df:pd.DataFrame):
    
    if 'future' in df.columns:
        df.drop('future', axis=1, inplace=True)
    
    for col in df.columns:
        if col != 'target': # normolize all columns, except the target
            # normalize each instrument. calculate relations based on percent change rather than value
            df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values) # Scale between 0 and 1
        
    df.dropna(inplace=True) # cleanup again, because nan may generate during percentage change ops



In [18]:
# df_temp = df.copy()
df = df_temp.copy()

In [19]:
preproc_data(df)

## Create sequences and Shuffle data

In [20]:
# Make data as timeseries

sequential_data = []
prev_days = deque(maxlen=period_len)

for i in df.values:
    prev_days.append([n for n in i[:-1]]) #keep all except the target
    if len(prev_days) == period_len:
        # insert every indivudual date prices, for period length days in this case 60 days
        # i[-1] is the target of 60th day target (last day of period)
        sequential_data.append([np.array(prev_days), i[-1]])


# now shuffle data
random.shuffle(sequential_data)

## Balance data with down sampling (buy/sell)

In [ ]:
buys = [] # to store buy data
sells = [] # to store sell data

for seq, target in sequential_data:
    if target == 0: # if its sell
        sells.append([seq, target]) 
    elif target == 1: # if its buy
        buys.append([seq, target])
        
random.shuffle(buys) # shuffle buy list
random.shuffle(sells) # shuffle buy list

        
        